In [2]:
import langchain
import langgraph
import langchain_core
import langchain_huggingface 
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint, HuggingFaceEmbeddings
import os 

from dotenv import load_dotenv
load_dotenv()
def create_model():
    
   hf_token=os.getenv("HF_TOKEN")
   if not hf_token:
        raise ValueError("the hf token is not available")   
   repo_id="Qwen/Qwen2.5-7B-Instruct"     
   llm=HuggingFaceEndpoint(
       repo_id=repo_id,
       huggingfacehub_api_token=hf_token,
       task="conversational"
   ) 
   model=ChatHuggingFace(llm=llm)

   return model 

from pydantic import BaseModel
from typing import Literal
class CognitiveOutput(BaseModel):
    state: Literal["overloaded", "normal", "underutilized", "fatigued"]
    action: Literal["reduce_task_load", "maintain", "increase_focus", "rest"]


In [ ]:
import json

async def cognitive_llm(inputs: dict):
    model = create_model()   # your LLM client

    prompt = f"""
You are a cognitive analyzer.

Based on the input metrics, return ONLY a JSON object
with keys "state" and "action".

Allowed states:
- overloaded
- normal
- underutilized
- fatigued

Allowed actions:
- reduce_task_load
- maintain
- increase_focus
- rest

Input metrics:
{json.dumps(inputs)}
"""

    response = await model.invoke(
      prompt
    )

    text = response.content

    # Parse & validate
    return CognitiveOutput.model_validate_json(text)



In [4]:
inputs = {
    "active_tasks": 7,
    "urgent_tasks": 3,
    "context_switches_last_hour": 12,
    "focus_minutes_today": 65,
    "sleep_hours_last_night": 5.5,
    "self_reported_stress": 7
}

result = await cognitive_llm(inputs)




c:\Users\hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'ChatHuggingFace' object has no attribute 'chat'

In [ ]:
print(result.model_dump())